# Guardamos las credenciales

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["QDRANT_URL"] = ""
os.environ["QDRANT_API_KEY"] = ""

In [ ]:
!pip install -U langchain-community
!pip install -U pypdf
!pip install -U python-dotenv

# Extrayendo chunks de un CSV



In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
loader = CSVLoader(
    file_path="/content/programadores (1).csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        # "fieldnames": ["Nombre", "Edad", "Tecnologías"],
    },
)

pages = loader.load()

In [ ]:
chunk_selector=1
print(f"{pages[chunk_selector].metadata}\n")
print(pages[chunk_selector].page_content)

{'source': '/content/programadores (1).csv', 'row': 1}

Nombre: Ana Gómez
Edad: 30
Tecnologias: Java, Kotlin


# Extrayendo chunks de un XLSX

In [ ]:
!pip install -U langchain-community
!pip install -U unstructured
!pip install -U openpyxl

In [ ]:
from langchain_community.document_loaders import UnstructuredExcelLoader


In [ ]:
loader = UnstructuredExcelLoader("/content/smex99 (1).xls", mode="elements")
pages = loader.load()

In [ ]:
chunk_selector=23
print(f"{pages[chunk_selector].metadata}\n")
print(pages[chunk_selector].page_content)

{'source': '/content/smex99 (1).xls', 'file_directory': '/content', 'filename': 'smex99 (1).xls', 'last_modified': '2025-01-30T10:40:28', 'page_name': 'Listado de variables', 'page_number': 2, 'languages': ['spa'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'category': 'Title', 'element_id': '16c3213d84db8a8f27781e0e59649f15'}

Emigraciones


# Extrayendo chunks de un PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

PDF Sevilla

In [ ]:
file_path = (
    "https://tic.visitasevilla.es/vsev/Web_php/imatges/guias/18189629ef0f8b27f2129128139.pdf"
)

In [ ]:
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    page.metadata["city"] = "Sevilla"
    pages.append(page)

PDF Málaga

In [ ]:
file_path = (
    "https://www.spain.info/export/sites/segtur/.content/Folletos/folletos/Malaga_ES.pdf"
)

In [ ]:
loader = PyPDFLoader(file_path)
async for page in loader.alazy_load():
    page.metadata["city"] = "Malaga"
    pages.append(page)

Vemos los chunks creados

In [ ]:
chunk_selector=12
print(f"{pages[chunk_selector].metadata}\n")
print(pages[chunk_selector].page_content)

{'source': 'https://tic.visitasevilla.es/vsev/Web_php/imatges/guias/18189629ef0f8b27f2129128139.pdf', 'page': 12, 'page_label': '13', 'city': 'Sevilla'}

/
Guía Práctica de Sevilla
ALOJAMIENTO
13
Legado Alcázar www.legadoalcazarhotel.com
Lobby Room & Suite www.lobbyroomhotel.com
M.A. Sevilla Congresos www.hotelsevillacongresos.com
Melia Lebreros www.melialebreros.solmelia.com
Melia Sevilla www.melia.com
Monte Triana www.hotelesmonte.com
Monte Carmelo www.hotelesmonte.com
NH Collection Sevilla www.nh-hoteles.es/hotel/nh-collection-sevilla
NH Plaza de Armas www.nh-hoteles.es/hotel/nh-sevilla-plaza-de-armas
NH Sevilla Viapol www.nh-hoteles.es/hotel/nh-sevilla-viapol
Novotel Sevilla www.novotel.com
One Shot Palacio Conde de Torrejón www.hoteloneshotpalaciocondetorrejon09.com
Palacio Alcázar www.palacioalcazar.com
Pasarela www.hotelpasarela.com
Petit Palace Marqués Santa Ana www.petitpalacemarquessantaana.com
Petit Palace Santa Cruz www.petitpalacesantacruzhotel.com
Palacio Pinello www.pala

# Indexando los chunks

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-qdrant

Seleccionamos el modelo de embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
import os
from langchain_qdrant import QdrantVectorStore

url = os.environ["QDRANT_URL"]
api_key = os.environ["QDRANT_API_KEY"]

qdrant = QdrantVectorStore.from_documents(
    documents=pages,
    embedding=embeddings,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="mi_primer_rag",
)